In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import Image, display
import torch
import glob

# Fix a seed for PyTorch
torch.manual_seed(4200);

## Enable GPU

gpu = True

if gpu == True:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
# Roboflow datasets are used in this notebook 
!pip install roboflow
from roboflow import Roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=52cfed0d38fa75a8ac88e1742f5e950f8472467bfc9d6917786ee0bb08d4eb14
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pypar

In [4]:
# Connect to Roboflow website using your account's workspace private API
rf = Roboflow(api_key="<your-roboflow-API-key>")

# link to the workspace of dataset wanted
project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")

# download the dataset for yolov5
dataset = project.version(6).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to American-Sign-Language-Letters-6 in yolov5pytorch:: 100%|██████████| 1452/1452 [00:01<00:00, 1331.62it/s]


In [5]:
# read a sample from the valid/images, for it's shape details and so on
import cv2

img = cv2.imread('/content/American-Sign-Language-Letters-6/valid/images/A10_jpg.rf.85a1d5108a4be1e892ed2d1eab546db9.jpg')
img.shape

(2047, 1536, 3)

In [6]:
# lets look at the data
!more /content/yolov5/American-Sign-Language-Letters-6/data.yaml

more: stat of /content/yolov5/American-Sign-Language-Letters-6/data.yaml failed: No such file or directory


In [7]:
# clone the yolov5 and install all the requirements

!git clone https://github.com/ultralytics/yolov5  # clone
%cd ./yolov5
%pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16008, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 16008 (delta 109), reused 129 (delta 83), pack-reused 15831
Receiving objects: 100% (16008/16008), 14.66 MiB | 20.76 MiB/s, done.
Resolving deltas: 100% (10978/10978), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.4/612.4 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [8]:
# move the dataset inside yolov5 (can also change the path used by default in the train.py)
!mv /content/American-Sign-Language-Letters-6 /content/yolov5/American-Sign-Language-Letters-6

In [10]:
# Track the weights and biases automatically using wandb 
# !pip install wandb
# !wandb login
# import wandb
# wandb.init(project="YOLOv5-ASL", entity="<your-entity-name>")

In [9]:
# best and last model weights are saved automatically and all the epochs results are also saved in runs/exp{number} folder of yolov5

# train the yolov5 model from scratch
# !python train.py --img 640 --batch 10 --epochs 50 --data /content/yolov5/American-Sign-Language-Letters-6/data.yaml --weights '' --cfg yolov5x.yaml --cache ram
# since the dataset is not that large, lets use transfer learning

# train the yolov5 model from pretrained weights
!python train.py --img 640 --batch 10 --epochs 50 --data /content/yolov5/American-Sign-Language-Letters-6/data.yaml --weights yolov5x.pt --cache ram

train: weights=yolov5x.pt, cfg=, data=/content/yolov5/American-Sign-Language-Letters-6/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_

In [11]:
# Zip the YOLOv5 folder in order to download it
# !zip -r /content/yolov5.zip /content/yolov5
!zip -r /content/yolov5x_pretrained.zip /content/yolov5/runs/train/exp


# and download manually:
#   * 'README.robotflow.txt'
#   * 'data.yaml'
#   * 'yolov5.zip'
#   * 'yolov5s.pt'
#   * this notebook

  adding: content/yolov5/runs/train/exp/ (stored 0%)
  adding: content/yolov5/runs/train/exp/confusion_matrix.png (deflated 15%)
  adding: content/yolov5/runs/train/exp/val_batch0_pred.jpg (deflated 10%)
  adding: content/yolov5/runs/train/exp/opt.yaml (deflated 48%)
  adding: content/yolov5/runs/train/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/yolov5/runs/train/exp/P_curve.png (deflated 10%)
  adding: content/yolov5/runs/train/exp/events.out.tfevents.1687864777.a950a6fc5192.938.0 (deflated 32%)
  adding: content/yolov5/runs/train/exp/results.png (deflated 8%)
  adding: content/yolov5/runs/train/exp/val_batch1_pred.jpg (deflated 8%)
  adding: content/yolov5/runs/train/exp/labels.jpg (deflated 34%)
  adding: content/yolov5/runs/train/exp/results.csv (deflated 82%)
  adding: content/yolov5/runs/train/exp/train_batch2.jpg (deflated 12%)
  adding: content/yolov5/runs/train/exp/val_batch0_labels.jpg (deflated 10%)
  adding: content/yolov5/runs/train/exp/train_batch0.jpg (de

In [ ]:
# from utils.plots import plot_results
# # plot 'results.csv' as 'results.png'
# plot_results('/content/yolov5/runs/train/exp10/results.csv') 
# # will already be there in the '/conent/yolov5/runs/train/exp10' as 'results.png'

In [ ]:
# # One feature that many people don't know about though is that you can still overlay multiple results by placing multiple results.csv files in a single directory dir and then:

# from utils.plots import *; plot_results(dir='path/to/dir')

In [ ]:
# !python train.py --resume

In [12]:
# save yolov5x.pt (weights after training and also include the model yolov5x.pb)
!python /content/yolov5/export.py --weights yolov5x.pt --include torchscript pb

export: data=data/coco128.yaml, weights=['yolov5x.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'pb']
YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients

PyTorch: starting from yolov5x.pt with output shape (1, 25200, 85) (166.0 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 19.2s, saved as yolov5x.torchscript (331.6 MB)

TensorFlow SavedModel: starting export with tensorflow 2.12.0...

                 from  n    params  module                                  arguments                     
2023-06-27 12:00:00.047711: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed ca

In [ ]:
# # include torchscript for the best model weights of all the epochs in the experiment (also weights are automatically saved in the yolov5/runs/exp folder)
# !python /content/yolov5/export.py --weights yolov5x.pt --include torchscript pb

In [ ]:
# copy the yolov5 folder to drive
!cp /content/yolov5 /content/drive/MyDrive/ML_Proj -r

In [ ]:
# !wandb offline

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
